In [24]:
!pip install openai

In [59]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
#from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
import requests
import json
import os

#### Nutritionix Request Test

In [60]:
url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'
headers = {
    'Content-Type': 'application/json',
    'x-app-id': os.environ['APPLICATION_ID'],
    'x-app-key':os.environ['APPLICATION_KEY'],
  #  'x-remote-user-id': 0
  }
body = {
    "query": "egg omelette with three eggs"
  }
response = requests.post(url=url, headers=headers, json=body)


In [61]:
response.json()

{'foods': [{'food_name': 'egg omelette',
   'brand_name': None,
   'serving_qty': 1,
   'serving_unit': '3-egg omelet',
   'serving_weight_grams': 178.38,
   'nf_calories': 323.47,
   'nf_total_fat': 25.32,
   'nf_saturated_fat': 5.86,
   'nf_cholesterol': 624.96,
   'nf_sodium': 533.35,
   'nf_total_carbohydrate': 1.39,
   'nf_dietary_fiber': 0.07,
   'nf_sugars': 0.62,
   'nf_protein': 21.13,
   'nf_potassium': 235.72,
   'nf_p': 333.09,
   'full_nutrients': [{'attr_id': 203, 'value': 21.1307},
    {'attr_id': 204, 'value': 25.3197},
    {'attr_id': 205, 'value': 1.3935},
    {'attr_id': 207, 'value': 2.5526},
    {'attr_id': 208, 'value': 323.47},
    {'attr_id': 210, 'value': 0.0001},
    {'attr_id': 211, 'value': 0.6223},
    {'attr_id': 212, 'value': 0.0007},
    {'attr_id': 213, 'value': 0},
    {'attr_id': 214, 'value': 0},
    {'attr_id': 221, 'value': 0},
    {'attr_id': 255, 'value': 127.9693},
    {'attr_id': 262, 'value': 0},
    {'attr_id': 263, 'value': 0},
    {'attr_id

#### LLM Open AI

In [81]:
from getpass import getpass

OPENAI_API_KEY = getpass()

In [82]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [128]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent

llm = ChatOpenAI()

### Agent Building

In [129]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

# class FoodSearchInput(BaseModel):
#     query: str = Field(description="description of a single food")

@tool
def FoodLookup(query: str) -> dict:
    "Function to lookup food nutrion facts"
    url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'
    headers = {
        'Content-Type': 'application/json',
        'x-app-id': os.environ['APPLICATION_ID'],
        'x-app-key':os.environ['APPLICATION_KEY'],
    #  'x-remote-user-id': 0
    }
    body = {
        "query": query
    }
    response = requests.post(url=url, headers=headers, json=body)
    return response.json()

# foodLookupTool = StructuredTool.from_function(
#     func=FoodLookup,
#     name="Food Lookup Tool",
#     description="lookup the nutrition information of a specfic food",
#     args_schema=FoodSearchInput,
#     # coroutine= ... <- you can specify an async method if desired as well
# )

In [130]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import MessagesPlaceholder

system_prompt = (
    "You are a helpful AI bot."
    "That is skilled at calculating calories and nutrion facts based on text input."
    "You have a tool that is able to find caloric information based on a string query."
    "Your job is to return the total calories and grams of protein consumed"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "Today I ate {food_query}. What is the total calories and protein in it?"),
    MessagesPlaceholder("agent_scratchpad")
    
])

tools = [FoodLookup]

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [131]:
agent_executor.invoke(input={'food_query': "One cup, Chobani Greek Yogurt, 1/2 cup Granola"})



> Entering new AgentExecutor chain...

Invoking: `FoodLookup` with `{'query': 'One cup Chobani Greek Yogurt'}`


{'foods': [{'food_name': 'greek yogurt', 'brand_name': None, 'serving_qty': 1, 'serving_unit': 'cup', 'serving_weight_grams': 226.67, 'nf_calories': 133.74, 'nf_total_fat': 0.88, 'nf_saturated_fat': 0.27, 'nf_cholesterol': 11.33, 'nf_sodium': 81.6, 'nf_total_carbohydrate': 8.16, 'nf_dietary_fiber': 0, 'nf_sugars': 7.34, 'nf_protein': 23.1, 'nf_potassium': 319.6, 'nf_p': 306, 'full_nutrients': [{'attr_id': 203, 'value': 23.0977}, {'attr_id': 204, 'value': 0.884}, {'attr_id': 205, 'value': 8.1601}, {'attr_id': 207, 'value': 1.632}, {'attr_id': 208, 'value': 133.7353}, {'attr_id': 210, 'value': 0}, {'attr_id': 211, 'value': 0}, {'attr_id': 212, 'value': 0}, {'attr_id': 213, 'value': 5.7574}, {'attr_id': 214, 'value': 0}, {'attr_id': 221, 'value': 0}, {'attr_id': 255, 'value': 192.8962}, {'attr_id': 262, 'value': 0}, {'attr_id': 263, 'value': 0}, {'attr_id': 268, 'value': 557.

{'food_query': 'One cup, Chobani Greek Yogurt, 1/2 cup Granola',
 'output': 'The total calories and protein in the One cup of Chobani Greek Yogurt and 1/2 cup of Granola are as follows:\n\n- Chobani Greek Yogurt (One cup):\n  - Calories: 133.74\n  - Protein: 23.1 grams\n\n- Granola (1/2 cup):\n  - Calories: 298.29\n  - Protein: 8.34 grams\n\nTotal Calories: 133.74 + 298.29 = 432.03 calories\nTotal Protein: 23.1 + 8.34 = 31.44 grams\n\nTherefore, the total calories consumed are 432.03 calories and the total protein consumed is 31.44 grams.'}